In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import TFAutoModelForMaskedLM, AutoModelForMaskedLM

model_checkpoint = "GroNLP/hateBERT"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

All PyTorch model weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [ ]:
model.summary()

Model: "tf_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108891648 
                                                                 
 mlm___cls (TFBertMLMHead)   multiple                  24459834  
                                                                 
Total params: 109514298 (417.76 MB)
Trainable params: 109514298 (417.76 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
text = "She works as a [MASK]."

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import numpy as np
import tensorflow as tf

inputs = tokenizer(text, return_tensors="np")
token_logits = model(**inputs).logits
mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = np.argsort(-mask_token_logits)[:5].tolist()

for token in top_5_tokens:
    print(f">>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

>>> She works as a model.
>>> She works as a trainer.
>>> She works as a consultant.
>>> She works as a dentist.
>>> She works as a baker.


In [ ]:
from datasets import load_dataset

gov_dataset = load_dataset("launch/gov_report")
gov_dataset

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'summary'],
        num_rows: 17519
    })
    validation: Dataset({
        features: ['id', 'document', 'summary'],
        num_rows: 974
    })
    test: Dataset({
        features: ['id', 'document', 'summary'],
        num_rows: 973
    })
})

In [ ]:
sample = gov_dataset["train"].shuffle(seed=42).select(range(3))

for row in sample:
    print(f"\n'>>> Review: {row['summary']}'")


'>>> Review: Critical infrastructure protection (CIP) activities called for in federal policy and law are intended to enhance the security of the public and private infrastructures that are essential to our nation's security, economic security, and public health and safety. Effective information-sharing partnerships between industry sectors and government can contribute to CIP efforts. Federal policy has encouraged the voluntary creation of information sharing and analysis centers (ISAC) to facilitate infrastructure sector participation in CIP information sharing efforts. GAO was asked to identify actions that the Department of Homeland Security (DHS) could take to improve the effectiveness of CIP information-sharing efforts. Federal awareness of the importance of securing the nation's critical infrastructures--and the federal government's strategy to encourage cooperative efforts among state and local governments and the private sector to protect these infrastructures--have been evol

In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["summary"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

tokenized_datasets = gov_dataset.map(
    tokenize_function, batched=True, remove_columns=['id', 'document', 'summary']
)
tokenized_datasets

Map:   0%|          | 0/17519 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (682 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/974 [00:00<?, ? examples/s]

Map:   0%|          | 0/973 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 17519
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 974
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
        num_rows: 973
    })
})

In [ ]:
tokenizer.model_max_length

512

In [ ]:
chunk_size = 128

In [ ]:
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 682'
'>>> Review 1 length: 524'
'>>> Review 2 length: 498'


In [ ]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 1704'


In [ ]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 40'


In [ ]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Map:   0%|          | 0/17519 [00:00<?, ? examples/s]

Map:   0%|          | 0/974 [00:00<?, ? examples/s]

Map:   0%|          | 0/973 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 81986
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 5294
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 5316
    })
})

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

"2 ) medical treatment policies and procedures for its deployed federal civilians. gao also examined the differences in special pays and benefits provided to dod's deployed federal civilians and military personnel. dod has established force health protection and surveillance policies to assess and reduce or prevent health risks for its deployed federal civilian personnel, but it lacks procedures to ensure implementation. our review of over 3, 400 deployment records at eight component locations found that components lacked documentation that some federal civilian personnel who deployed to afghanistan and iraq had received, among other things, required pre - and post - deployment health assessments and immunizations. these deficiencies were most prevalent at air force and navy"

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] as [MASK] department of defense ( dod ) has expanded its involvement in overseas military operations, it has grown increasingly reliant on its federal civilian workforce to support contingency [MASK]. the senate armed services committee required gao [MASK] [MASK] dod's policies concerning [MASK] health care for dod civilians who deploy in support of conting [MASK] operations illustrating [MASK] and iraq. gao analyzed over [MASK], 400 deployment - [MASK] records for deployed [MASK] civilians and interviewed [MASK] officials to determine the extent to which dod has established and the [MASK] [MASK] and defense agencies ( hereiclester referred to as dod components [MASK] have implemented ( 1 ) force health protection and surveillance [MASK] and ('

'>>> 2 ) medical treatment policies [MASK] procedures influential its deployed federal [MASK]. [MASK] also examined the differences in special pays and vintage provided to dod's [MASK] federal civilians and military personnel. dod h

In [ ]:
import collections
import numpy as np

from transformers.data.data_collator import tf_default_data_collator

wwm_probability = 0.2


def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
        feature["labels"] = new_labels

    return tf_default_data_collator(features)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] as the department of defense ( dod [MASK] [MASK] expanded its [MASK] in overseas military operations, it has [MASK] increasingly reliant on its federal [MASK] [MASK] to support contingency operations. the senate armed [MASK] committee [MASK] gao to [MASK] [MASK] [MASK] s policies concerning [MASK] [MASK] care for dod civilians who deploy in support of contingency operations [MASK] afghanistan [MASK] iraq [MASK] gao [MASK] [MASK] 3, 400 deployment - [MASK] records for deployed [MASK] civilians and [MASK] department officials to determine the extent to which [MASK] has established and [MASK] military services [MASK] defense agencies [MASK] hereafter referred to as dod components ) [MASK] [MASK] ( 1 [MASK] force health protection [MASK] surveillance [MASK] [MASK] ('

'>>> 2 ) [MASK] treatment policies and procedures for its deployed federal civilians. gao also examined the [MASK] in special pays [MASK] benefits provided [MASK] dod'[MASK] [MASK] federal civilians and military p

In [ ]:
train_size = 30_000
test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 30000
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 3000
    })
})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
tf_train_dataset = model.prepare_tf_dataset(
    downsampled_dataset["train"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=32,
)

tf_eval_dataset = model.prepare_tf_dataset(
    downsampled_dataset["test"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=32,
)

In [ ]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

tf.keras.mixed_precision.set_global_policy("mixed_float16")

model_name = model_checkpoint.split("/")[-1]
callback = PushToHubCallback(
    output_dir=f"{model_name}-finetuned-govreport", tokenizer=tokenizer
)

Cloning https://huggingface.co/viditnaik/hateBERT-finetuned-govreport into local empty directory.


In [ ]:
import math

eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

94/94 [==============================] - 45s 354ms/step - loss: 5.4272
Perplexity: 227.51


In [ ]:
model.fit(tf_train_dataset, validation_data=tf_eval_dataset, callbacks=[callback])

937/937 [==============================] - 1029s 1s/step - loss: 3.3500 - val_loss: 2.4926


In [ ]:
eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

94/94 [==============================] - 34s 357ms/step - loss: 2.4958
Perplexity: 12.13


In [ ]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="viditnaik/hateBERT-finetuned-snli"
)

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at viditnaik/hateBERT-finetuned-snli.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [ ]:
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> she works as a nurse.
>>> she works as a trainer.
>>> she works as a model.
>>> she works as a photographer.
>>> she works as a waitress.
